In [1]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from node2vec import Node2Vec

# Load the KG

In [2]:
path = '../Datasets/MetaQA_dataset/'

df = pd.read_csv(path+'kb.txt', sep='|', header=None, names=['entity1', 'relation', 'entity2'])
df.head()

,entity1,relation,entity2
0,Kismet,directed_by,William Dieterle
1,Kismet,written_by,Edward Knoblock
2,Kismet,starred_actors,Marlene Dietrich
3,Kismet,starred_actors,Edward Arnold
4,Kismet,starred_actors,Ronald Colman


In [3]:
df_unique = df.drop_duplicates()
len(df_unique)

133582

Note: The NetworkX graph is constructed using edges only and does not account for nodes without links (as present in kb_entity_dict.txt). I'm assuming these isolated nodes do not provide useful embeddings based on graph topology.

In [4]:
# Use MultiGraph to allow multiple edges
G = nx.from_pandas_edgelist(df_unique, source='entity1', target='entity2', edge_attr='relation', create_using=nx.MultiGraph())

In [5]:
num_nodes = G.number_of_nodes()
print(f"Number of entities: {num_nodes}")

num_edges = G.number_of_edges()
print(f"Number of edges: {num_edges}")

distinct_relations = set(nx.get_edge_attributes(G, 'relation').values())
print(f"Number of distinct relations: {len(distinct_relations)}")
print("Distinct relations:", distinct_relations)

Number of entities: 43234
Number of edges: 133582
Number of distinct relations: 9
Distinct relations: {'starred_actors', 'has_genre', 'has_imdb_votes', 'written_by', 'has_imdb_rating', 'release_year', 'has_tags', 'in_language', 'directed_by'}


# Node2Vec Embeddings

In [6]:
# Create Node2Vec model from the graph
node2vec = Node2Vec(G, dimensions=64, walk_length=10, num_walks=100, workers=4)

# Fit model to generate node embeddings
model = node2vec.fit(window=10, min_count=1, batch_words=4)

Computing transition probabilities:   0%|          | 0/43234 [00:00<?, ?it/s]

In [7]:
# Save embeddings for later use
model.wv.save_word2vec_format('ud_node2vec_embeddings.txt')

# Save model for later use
model.save('ud_node2vec_model.model')

# Load the saved model

In [8]:
from gensim.models import Word2Vec

In [11]:
# model = Word2Vec.load('ud_node2vec_model.model')
model = Word2Vec.load('d_node2vec_model.model')

In [12]:
# Look for most similar nodes
query_node = 'Top Hat'
model.wv.most_similar(query_node)

[('Dames', 0.9805895686149597),
 ('Swing Time', 0.9764919877052307),
 ("It's a Great Feeling", 0.9735110402107239),
 ('A Funny Thing Happened on the Way to the Forum', 0.9711706042289734),
 ('Bye Bye Birdie', 0.969506561756134),
 ('Mr. Blandings Builds His Dream House', 0.9674002528190613),
 ('The Band Wagon', 0.9672629237174988),
 ('Ruggles of Red Gap', 0.9665829539299011),
 ('Anchors Aweigh', 0.9660155177116394),
 ('My Favorite Brunette', 0.9654163718223572)]

# TO DO: 
# 1. Use embeddings to train model for QA
# - need to extract keywords from the question